# SUDS to MiniSEED single event converter for Pinatubo data
This Python3 notebook demonstrates how to convert a PC-SUDS seismic waveform data file into Miniseed
It requires Win-SUDS Utilities (for programs demux.exe, irig.exe and sud2sac.exe) and ObsPy. However, there is a new capability to read DMX files directly into ObsPy. By comparing that with results of loading SAC files from sud2sac.exe, I've figured out how to wrap the DMX reader to end up with the same data array, through subtracting 2048 and converting from uint16 to float, and removing 'unk' from network code. However, sample rates are still subtley different and I don't know what correct value is. DMX reader gets ~100.8 Hz, SAC files have 100.6-100.9, and presumably the clock was set to 100 Hz but not accurate. 

Do all library imports first

In [ ]:
import obspy.core as op
import glob
import matplotlib.pyplot as plt
import os
import shutil

Set up all paths that will be used

In [ ]:
os.chdir('C:\DATA\Pinatubo')

WINSUDSPATH = os.path.join('C:\\', 'winsuds','bin')
#SUDSbasename = 'waveforms/May91/9105010W'
WAVEFORM_DIR = 'WAVEFORMS'
CONVERT_DIR = 'convert'
#originalWaveformFile = os.path.join(WAVEFORM_DIR, 'May91', '9105011P.DMX')
originalWaveformFile = os.path.join(WAVEFORM_DIR, 'Dec91', '9112010B.DMX')
DMXbasename = os.path.basename(originalWaveformFile)
DMXfile = os.path.join(CONVERT_DIR, 'original',DMXbasename) # original

IRIGfile = os.path.join(CONVERT_DIR, 'irig', DMXbasename) # produced by irig.exe
IRIGcopy = os.path.join(CONVERT_DIR, 'sac', DMXbasename)
SACbasename = os.path.join(CONVERT_DIR, 'sac', DMXbasename.replace('.DMX', '.sac')) # produced  by sud2sac.exe
MSEEDfile0 = os.path.join(CONVERT_DIR, 'original', DMXbasename.replace('.DMX', '.ms')) # produced by sud2msed.exe
MSEEDfile1 = os.path.join(CONVERT_DIR, 'irig', DMXbasename.replace('.DMX', '.ms')) # produced by sud2msed.exe
MSEEDfile2 = os.path.join(CONVERT_DIR, 'sac', DMXbasename.replace('.DMX', '.mseed')) # produced by recombining SAC file
PHAfile = SUDSbasename + '.PHA' # this might exist if HYPO71 was run to locate the event
#PUNfile = SUDSbasename + '.PUN' # this might exist if HYPO71 was run and generated a hypocenter
demux = os.path.join(WINSUDSPATH, 'demux.exe')
irig = os.path.join(WINSUDSPATH, 'irig.exe')
sud2sac = os.path.join(WINSUDSPATH, 'sud2sac.exe')
sud2msed = os.path.join(WINSUDSPATH, 'sud2msed.exe')
sudsplot = os.path.join(WINSUDSPATH, 'sudsplot.exe')
print('Paths setup okay')

def read_DMX_file(DMXfile):
    # I have found this produces same result as converting DMX to SAC with sud2sac.exe in Win-SUDS, and then reading into ObsPy
    # Tested only on Pinatubo 1991 data
    # DMXfile is normally read in as uint16 and so is all +ve. But SUDS data must actually be 2s-complement or sign bit, because
    # sud2sac.exe converts to numbers either size of 0. Difference always seems to be 2048. 
    # Obspy Miniseed writer needs float, not int, so will crash unless recast as float.
    # However, sampling rates between DMX and SAC are different
    st = op.read(DMXfile)
    for tr in st:
        if tr.stats.station=='IRIG':
            st.remove(tr) # we do not want to keep the IRIG trace
        if all(tr.data==0):
            st.remove(tr)
        if tr.stats.network == 'unk':
            tr.stats.network = ''
        tr.data = tr.data.astype(float) - 2048.0     
    return st


Check that orignal DMX file can be read by ObsPy

In [ ]:
shutil.copyfile(originalWaveformFile, DMXfile)

try:
    print(DMXfile)
    st = read_DMX_file(DMXfile)
    print('- read okay')
    print(st) 
    try:
        print('Writing DMX file data to %s with ObsPy' % MSEEDfile0)
        st.write(MSEEDfile0, 'mseed')
        print('- Success')
    except:
        print('- FAILED')    
except:
    print('- ObsPy cannot read this demultiplexed SUDS format')

Time correct the DMX file and convert to SAC files. 

Then read in, plot, and attempt top write to MiniSEED.

Note: DMX read support now (2023) included in ObsPy. Was not available for the Montserrat ASN conversion in 2019.

In [ ]:

if os.path.exists(DMXfile): # if DMX file exists, time correct it then convert to SAC
    if not os.path.exists(IRIGfile):
        shutil.copyfile(DMXfile, IRIGfile)
        print('Time correcting ' + IRIGfile)
        os.system(irig + ' ' + IRIGfile)
    
    print('Reading ' + IRIGfile)
    try:
        st2 = read_DMX_file(IRIGfile)
        print('- Success')
        for tr in st:
            tr.plot();
    except:
        print('- FAILED')
    else:
        print(st2) 
        try:
            print('Writing DMX-IRIG file data to %s with ObsPy' % MSEEDfile1)
            st2.write(MSEEDfile1, 'mseed')
            print('- Success')
        except:
            print('- FAILED')
    

else:
    print(DMXfile + ' does not exist')
    

Convert IRIG DMX file to SAC files with sud2sac.exe

Note that we found that sud2msed does not read trace headers, it just produces one trace from many and also has an incorrect start time. This is why we use sud2sac, which we have checked against sudsplot.exe and shows the correct trace headers, waveforms and times.

Merge the SAC files into a single Miniseed file. 

In [ ]:
print(os.getcwd())
if os.path.exists(IRIGfile): 
    # ALTERNATIVE: Rather than reading into ObsPy and saving to Miniseed
    # convert to sac first, then read in. And compare.
    # This produces one file per trace
    print('Converting ' + IRIGfile + ' to SAC files')
    if not os.path.exists(IRIGcopy):
        shutil.copy(IRIGfile, IRIGcopy)
    print(sud2sac + ' '  + IRIGcopy)    
    os.system(sud2sac + ' ' + IRIGcopy)
    
    # Now merge the SAC files into a single valid Miniseed file    
    print('Reading from SAC files')
    st3 = op.Stream()
    sacfilelist = glob.glob(SACbasename + '-???')
    print(sacfilelist)
    if len(sacfilelist) > 0:
        for sacfile in sacfilelist:
            print('- Reading ' + sacfile)
            try:
                sacst = op.read(sacfile)   
                #tr.plot();
            except:
                print('  - FAILED')
            else:
                for tr in sacst:
                    tr2 = tr.copy().detrend()
                    if not (all(tr2.data==0)):
                        st3 = st3 + tr
        print(st3)
        #st3.plot(equal_scale=False);
        
        print('Writing SAC file data to %s with ObsPy' % MSEEDfile2)
        try:
            st3.write(MSEEDfile2, 'mseed')
            print('- Success')
        except:
            print('- FAILED')
    else:
        print('FAILED. No SAC files found matching ' + SACbasename)
else:
    print(IRIGfile + ' NOT FOUND')

Compare all 3 MiniSEED files. Are they the same?

In [ ]:
st0 = op.read(MSEEDfile0)
st1 = op.read(MSEEDfile1)
st2 = op.read(MSEEDfile2)

def compare_Streams(st0, st1):
    ids0 = [tr.id for tr in st0]
    ids1 = [tr.id for tr in st1]
    print('trace ID lists are same')
    print(' ')
    if not ((ids0==ids1) and (ids0==ids2)):
        print('trace ID lists are different')
        print(ids0)
        print(ids1)        
    for i,tr0 in enumerate(st0):
        okay = True
        if not tr0.stats.npts == st1[i].stats.npts:
            print("different number of samples") 
            print(tr0.stats.npts, st1[i].stats.npts) 
            okay = False        
        if not tr0.stats.sampling_rate == st1[i].stats.sampling_rate:
            print("different sampling rates") 
            print(tr0.stats.sampling_rate, st1[i].stats.sampling_rate) 
            okay = False
        if not (all(tr0.times()==st1[i].times())):
            print("different time array")   
            okay = False
            if not (tr0.stats.starttime==st1[i].stats.starttime):
                print('different start times')
                print(tr0.stats.starttime, st1[i].stats.starttime)
            if not (tr0.times()[-1]==st1[i].times()[-1]):
                print('dfferent end times')
                print(tr0.times()[-1], st1[i].times()[-1])
        if not (all(tr0.data==st1[i].data)):
            print("different data array")
            same=(tr0.data==st1[i].data)
            diff1=st1[i].data - tr0.data
            print('Out of %d samples, %d same' % (tr0.stats.npts, same.sum())
            print('Differences: max: %f, mean: %f' % (np.max(diff1), np.mean(diff1)))
            okay = False    
        if okay:
            print(tr0.id,' is okay')
        else:
            print(tr0.id,' FAILED')
            tr0.plot();
            st1[i].plot();
        print(' ')    
                  
compare_streams(st0, st1)
compare_streams(st0, st2)

In [ ]:
# SUDSplot can be used to compare if Miniseed conversion was accurate
os.system(sudsplot + ' ' + DMXfile)

Add this Miniseed event waveform file to the Seisan database.
The Miniseed file will be moved to [SEISANTOP]/WAV/[SEISANDB]/YYYY/MM
A corresponding Seisan S-file (event metadata file) will be created at [SEISANTOP]/REA/[SEISANDB]/YYYY/MM

The Seisan programs MAKEREA and AUTOREG are used here. Since they normally require user input, we create files to simulate this.

In [ ]:
seisanDBname = 'PNTBO'
yy = DMXbasename[0:2]
mm = DMXbasename[2:4]
century = '19'
if yy < '80':
    century = '20'
yyyy = century + yy
fptr = open('makerea_wrapper.txt','w')
fptr.write(seisanDBname + '\n')
fptr.write(yyyy + mm + '\n')
fptr.write('\n')
fptr.write('BOTH\n')
fptr.close()
os.system('makerea < makerea_wrapper.txt')

Copy the miniseed file to WOR BEFORE running dirf and autoreg

In [ ]:
WORpath = os.path.join(os.getenv('SEISAN_TOP'),'WOR')
WORfile = os.path.join(WORpath, os.path.basename(MSEEDfile2))
cwd=os.getcwd()
print('cwd = ' + cwd)
shutil.copyfile(MSEEDfile2, WORfile)

os.chdir(WORpath)
os.system('dirf ' + os.path.basename(WORfile))
fptr = open('autoreg_wrapper.txt','w')
fptr.write('L\n')
fptr.write('m\n')
fptr.write(seisanDBname + '\n')
fptr.write('gt\n')
fptr.close()
os.system('autoreg < autoreg_wrapper.txt')
os.chdir(cwd)

Can now browse event with:
    eev 199110 PNTBO
    
Can also view the latest S-File with the code below:

In [ ]:
def findLatestFile(dirpath):
    lists = os.listdir(dirpath)                                   
    lists.sort(key=lambda fn:os.path.getmtime(os.path.join(dirpath, fn)))
    return os.path.join(dirpath,lists[-1])

def displayFile(dirpath):
    if os.path.exists(dirpath):
        fptr = open(dirpath, 'r')
        str = fptr.read()
        print(str)
    else:
        print(dirpath + ' not found')

SFILE = findLatestFile(os.path.join(os.environ['SEISAN_TOP'],'REA',seisanDBname,yyyy,mm))
print(SFILE)
displayFile(SFILE)